In [1]:
# Keras
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping # コールバック関数
from keras.optimizers import SGD # 最適化関数
from keras.initializers import glorot_uniform, orthogonal, TruncatedNormal # 初期化関数
from keras.preprocessing.image import load_img, array_to_img, img_to_array # 画像の取り扱い
from keras.utils import np_utils, plot_model # ?
from keras.layers import Dense, Activation, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, BatchNormalization # レイヤー
from keras.layers.advanced_activations import PReLU # 応用活性化関数
from keras.layers.recurrent import GRU, SimpleRNN # RNN系関数
from keras import backend as K

# ファインチューニング
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# 計算
import pandas as pd #行列計算
import numpy as np #行列計算
import math #数値計算
import itertools #順列・組み合わせ

# 図・画像
import matplotlib.pyplot as plt #グラフ
import seaborn as sns
from PIL import Image, ImageFilter

# 金融
import mpl_finance as mpf
import talib as ta # テクニカル指標

# scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error


# GBDT
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostClassifier, Pool

# フォント
import matplotlib.font_manager as fm
from matplotlib import rcParams

# その他
import glob
import re
import gc
import cv2
import os
from datetime import datetime, timedelta

# マルチプロセス・マルチスレッド
import threading
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 512
EPOCHS = 100
VERBOSE = 1
VALIDATION_SPLIT = 0.2
MODEL = 'CNN'#'ResNet50'#'CNN' #'MyVGG16'
IMG_CHANNELS = 3
#KAITEN_MIZUMASHI = #range(0, 360, 30) # range(0, 1, 90)
KAITEN_MIZUMASHI = range(0, 1, 90)

In [3]:
# 余白を追加
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [4]:
# 余白を追加して正方形に変形
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [5]:
img_size = 128
X = []
y = []

#img_path_list_1 = glob.glob('image/ddh-01/*')
#img_path_list_2 = glob.glob('image/ddh-10/*')
#img_path_list_1 = glob.glob('image/ddh-01/*')
#img_path_list_2 = glob.glob('image/ddh-10/*')

img_path_list_1 = glob.glob('image/ddh-01_under_50000/*')
img_path_list_2 = glob.glob('image/ddh-10_under_50000/*')
img_path_list_3 = glob.glob('image/ddh-06_under_50000/*')
img_path_list_4 = glob.glob('image/ddh-07_under_50000/*')
img_path_list_5 = glob.glob('image/ddh-09_under_50000/*')

#img_path_list_1 = glob.glob('image/ddh-01_face_under_50000/*')
#img_path_list_2 = glob.glob('image/ddh-10_face_under_50000/*')
img_path_list = img_path_list_1# + img_path_list_2# + img_path_list_3 + img_path_list_4

for path in img_path_list[:100]:
    print(path)
    img = Image.open(path) # 開いて
    print(type(img))
    img = expand2square(img, (255, 255, 255)) # 正方形に整形して
    img = img.resize((img_size, img_size)) # サイズを調整する
    
    # グレースケールに変換
    if IMG_CHANNELS == 1:
        img = img.convert('L')
        img = np.array(img)
        img = img.reshape(img_size, img_size, 1) # データの形式を変更
        X.append(img) # 学習データ
    else:
        X.append(np.array(img)) # 学習データ
        
    y.append(int(re.search(r'[0-9]{8,8}', path).group(0))) # ラベルデータ
    
# numpy配列に変換
X = np.array(X)
y = np.array(y)

image/ddh-01_under_50000\00001000_bff6ee59-407e-4553-bfbb-264782fdbb4c.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00001200_fd06596e-0569-4fe5-a1f9-8969dad054bd.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002000_73b24590-c7c3-480f-a7fa-e2b0c36d73d2.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002000_a1cea8ae-8055-4064-86ab-bde78e1a3e0d.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002000_d62c7d0b-a133-4601-9fbe-cab22953046f.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002000_ee340ea2-8e3f-435f-9260-5e52aa968061.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002600_e97c873b-517f-45ef-b931-7dfcce04fb9a.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002800_01ab0b7c-f30b-49d4-be07-fcc58f887a01.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00002800_60bb53

image/ddh-01_under_50000\00005000_1b2d8c0d-d355-4646-aaa6-de3280f5efa7.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_1bb4652a-b97f-43ae-9173-363f5de6aecc.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_20d644b8-18d9-4b07-a065-5f6fc8dc5588.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_22dfc0fa-a965-473c-af20-6f7c81441b77.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_246ccd78-ed1c-4864-a509-d4d1888a96c9.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_31c87ead-f187-405c-bc52-369a67c0f6a0.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_3280c244-7ef2-4edb-a427-bab32b44b774.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_32f308b8-56c6-405f-a3f7-3193f80c6edf.jpeg
<class 'PIL.JpegImagePlugin.JpegImageFile'>
image/ddh-01_under_50000\00005000_345469

In [6]:
# numpy配列に変換
X_ = np.array(X)
y_ = np.array(y)

In [7]:
model = load_model('model_doll_cnn', compile=False)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [8]:
pred = model.predict(X_).flatten()

df_pred = pd.DataFrame([pred, y_])
df_pred = df_pred.T
df_pred.to_csv("pred.csv", index=False)

df_pred

,0,1
0,14584.973633,1000.0
1,11103.952148,1200.0
2,13097.740234,2000.0
3,13730.284180,2000.0
4,9432.926758,2000.0
...,...,...
95,14693.092773,5000.0
96,6879.964844,5000.0
97,13253.140625,5000.0
98,16438.035156,5000.0
